In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile data_json_csv.py

import argparse
import collections
import csv
import json

def read_and_write_file(json_file_path, csv_file_path, column_names):

    with open(csv_file_path, 'w+') as fout:
        csv_file = csv.writer(fout)
        csv_file.writerow(list(column_names))
        with open(json_file_path) as fin:
            for line in fin:
                line_contents = json.loads(line)
                csv_file.writerow(get_row(line_contents, column_names))

def get_superset_of_column_names_from_file(json_file_path):
    column_names = set()
    with open(json_file_path) as fin:
        for line in fin:
            line_contents = json.loads(line)
            column_names.update(
                    set(get_column_names(line_contents).keys())
                    )
    return column_names

def get_column_names(line_contents, parent_key=''):
    
    """Return a list of flattened key names given a dict.
    Example:
        line_contents = {
                'a': {
                    'b': 2,
                    'c': 3,
                    },
        }
        will return: ['a.b', 'a.c']
    These will be the column names for the eventual csv file.
    """
    column_names = []
    for k, v in line_contents.items():
        column_name = "{0}.{1}".format(parent_key, k) if parent_key else k
        if isinstance(v, collections.abc.MutableMapping):
            # if parent key is not in the column_names, add parent key,value pair first
            if column_name not in column_names:
                column_names.append((column_name,v))
            # use recursive call to flatten the nested dictionaries
            column_names.extend(get_column_names(v, column_name).items())
        else:
            column_names.append((column_name, v))
    return dict(column_names)

def get_nested_value(d, key):
    """Return a dictionary item given a dictionary d and a flattened key from get_column_names.
    Example:
        d = {
            'a': {
                'b': 2,
                'c': 3,
                },
        }
        key = 'a.b'
        will return: 2
    """
    if '.' not in key:
        if key not in d:
            return None
        return d[key]
    base_key, sub_key = key.split('.', 1)
    if base_key not in d:
        return None
    sub_dict = d[base_key]
    if isinstance(sub_dict, dict):
        return get_nested_value(sub_dict, sub_key)
    elif sub_dict is None:
        return None
    else:
        print(sub_dict)
        return None

def get_row(line_contents, column_names):
    """Return a csv compatible row given column names and a dict."""
    row = []
    for column_name in column_names:
        line_value = get_nested_value(
                        line_contents,
                        column_name,
                        )

        if line_value is not None:
            row.append('{0}'.format(line_value))
        else:
            row.append('')
    return row

if __name__ == '__main__':

    parser = argparse.ArgumentParser(
            description='Convert file from json to csv',
            )

    parser.add_argument(
            'json_file',
            type=str
            )

    args = parser.parse_args()

    json_file = args.json_file
    csv_file = '{0}.csv'.format(json_file.split('.json')[0])

    column_names = get_superset_of_column_names_from_file(json_file)
    read_and_write_file(json_file, csv_file, column_names)

Writing data_json_csv.py


In [ ]:
%run -i 'data_json_csv.py' '/content/drive/MyDrive/YelpRecommender/Data/yelp_academic_dataset_business.json'

In [ ]:
%run -i 'data_json_csv.py' '/content/drive/MyDrive/YelpRecommender/Data/yelp_academic_dataset_user.json'

In [ ]:
%run -i 'data_json_csv.py' '/content/drive/MyDrive/YelpRecommender/Data/yelp_academic_dataset_review.json'
%run -i 'data_json_csv.py' '/content/drive/MyDrive/YelpRecommender/Data/yelp_academic_dataset_tip.json'
%run -i 'data_json_csv.py' '/content/drive/MyDrive/YelpRecommender/Data/yelp_academic_dataset_checkin.json'

In [ ]:
reviews = pd.read_csv('/content/drive/MyDrive/YelpRecommender/Data/review.csv')

In [ ]:
reviews.head()

,date,user_id,useful,cool,text,funny,review_id,business_id,stars
0,2018-07-07 22:09:11,mh_-eMZ6K5RLWhZyISBhwA,0,0,"If you decide to eat here, just be aware it is...",0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3.0
1,2012-01-03 15:28:18,OyoGAe7OKpv6SyGZT5g77Q,1,1,I've taken a lot of spin classes over the year...,0,BiTunyQ73aT9WBnpR9DZGw,7ATYjTIgM3jUlt4UM3IypQ,5.0
2,2014-02-05 20:30:30,8g_iMtfSiwikVnbP2etR0A,0,0,Family diner. Had the buffet. Eclectic assortm...,0,saUsX_uimxRlCVr67Z4Jig,YjUWPpI6HXG530lwP-fb2A,3.0
3,2015-01-04 00:01:03,_7bHUi9Uuf5__HHc_Q8guQ,1,1,"Wow! Yummy, different, delicious. Our favo...",0,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5.0
4,2017-01-14 20:54:15,bcjbaE6dDog4jkNY91ncLQ,1,1,Cute interior and owner (?) gave us tour of up...,0,Sx8TMOWLNuJBWer-0pcmoA,e4Vwtrqf-wpJfwesgvdgxQ,4.0


In [2]:
import pandas as pd

In [3]:
business = pd.read_csv('/content/drive/MyDrive/YelpRecommender/Data/business.csv',low_memory=False)

In [ ]:
len(business)

150346

In [4]:
ziplen = business.postal_code.astype(str).apply(len) # drop entries where postal_code is not 5-digit from business
business = business[ziplen == 5]


In [5]:
states = ['LA']
business = business[business.state.isin(states)]

In [6]:
len(business)

9917

In [15]:
business.categories.head(20)

17     Vape Shops, Tobacco Shops, Personal Shopping, ...
18     Automotive, Car Rental, Hotels & Travel, Truck...
34     Event Planning & Services, Hotels, Hotels & Tr...
60                   Restaurants, Soup, Seafood, Burgers
65                  Hotels & Travel, Tours, Local Flavor
87     Nightlife, Pubs, Event Planning & Services, Wi...
96     Event Planning & Services, Caterers, Party & E...
103                   Restaurants, Seafood, Cajun/Creole
118    Flowers & Gifts, Embroidery & Crochet, Uniform...
127    Tires, Auto Repair, Oil Change Stations, Autom...
131    Cocktail Bars, Italian, Nightlife, Seafood, Ba...
155    Hotels, Hotels & Travel, Bed & Breakfast, Even...
189        Restaurants, Sandwiches, Pizza, Chicken Wings
231    American (New), Food, Bars, Nightlife, Lounges...
253      Pizza, Restaurants, Bars, Nightlife, Sandwiches
255    Fitness & Instruction, Gyms, Weight Loss Cente...
265    Diagnostic Imaging, Diagnostic Services, Healt...
267                    Fashion,

In [7]:
for i in business.categories:
  if not isinstance(i, str):
    print(i,type(i))

nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>


In [ ]:
#has a lot of nans of type float we ignore them.

In [8]:
categories_explode = lambda x: [a.strip().lower() for a in x.split(',')] if isinstance(x, str) else print(x)

In [9]:
categories = business.categories.apply(categories_explode)

nan
nan
nan
nan
nan
nan
nan
nan


In [10]:
df = pd.DataFrame(categories.explode('categories'))

In [13]:
df.groupby('categories').value_counts().sort_values(ascending=False)[:50]

categories
restaurants                  3638
food                         1994
shopping                     1639
nightlife                    1162
bars                         1031
event planning & services     901
beauty & spas                 800
hotels & travel               779
arts & entertainment          616
sandwiches                    589
automotive                    587
local services                568
seafood                       550
home services                 545
health & medical              534
breakfast & brunch            526
cajun/creole                  524
american (traditional)        500
coffee & tea                  472
fashion                       461
american (new)                461
active life                   437
burgers                       400
hotels                        377
specialty food                350
fast food                     338
home & garden                 330
pizza                         293
nail salons                   270
loc

In [12]:
len(df.groupby('categories').value_counts().sort_values(ascending=False))

983

In [16]:
# There are a total of 983 total categories of businesses in LA.
# Now we need to select only restaurant based businesses and ignore rest
rest_categories = ['restaurants','food','nightlife','bars','sandwiches','seafood','breakfast & brunch','cajun/creole','coffee & tea '\
 ,'burgers','specialty food','fast food','pizza','local flavor','cafes','cocktail bars','desserts','mexican','chicken wings'\
 ,'italian','bakeries','salad','ice cream & frozen yogurt']

In [22]:
restaurant_indexes = []
other_indexes = []
for idx,row in business.iterrows():
  # print(idx,row['categories'])
  if isinstance(row['categories'], str):
    for cat in row['categories'].split(','):
      if cat.strip().lower() in rest_categories:
        restaurant_indexes.append(idx)
        break
      else:
        other_indexes.append(idx)


In [29]:
rest_indexes = set(other_indexes) - set(restaurant_indexes)

In [37]:
business_restaurants = business.drop(index=rest_indexes)

In [38]:
len(business_restaurants)

5104

In [39]:
business_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5104 entries, 60 to 150283
Data columns (total 60 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   state                                  5104 non-null   object 
 1   attributes.RestaurantsPriceRange2      4274 non-null   object 
 2   attributes.RestaurantsCounterService   1 non-null      object 
 3   attributes.DietaryRestrictions         1 non-null      object 
 4   attributes.BestNights                  579 non-null    object 
 5   attributes.HairSpecializesIn           0 non-null      object 
 6   attributes.Ambience                    3389 non-null   object 
 7   attributes.NoiseLevel                  2891 non-null   object 
 8   attributes.Smoking                     405 non-null    object 
 9   attributes.GoodForMeal                 2151 non-null   object 
 10  hours.Tuesday                          3689 non-null   object 
 11  a

In [49]:
new_orleans_business = business_restaurants[(business_restaurants['city']=='New Orleans') | (business_restaurants['city']=='NEW ORLEANS')]

In [51]:
len(new_orleans_business)

3306

In [54]:
new_orleans_business.to_csv('/content/drive/MyDrive/YelpRecommender/Data/NewOrleans/business.csv')

In [40]:
user = pd.read_csv('/content/drive/MyDrive/YelpRecommender/Data/user.csv')

In [41]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   yelping_since       object 
 1   compliment_cool     int64  
 2   compliment_profile  int64  
 3   compliment_photos   int64  
 4   funny               int64  
 5   compliment_more     int64  
 6   elite               object 
 7   compliment_list     int64  
 8   user_id             object 
 9   compliment_writer   int64  
 10  useful              int64  
 11  review_count        int64  
 12  compliment_plain    int64  
 13  compliment_hot      int64  
 14  average_stars       float64
 15  compliment_funny    int64  
 16  compliment_note     int64  
 17  cool                int64  
 18  name                object 
 19  friends             object 
 20  compliment_cute     int64  
 21  fans                int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 333.7+ MB


In [42]:
user.head()

,yelping_since,compliment_cool,compliment_profile,compliment_photos,funny,compliment_more,elite,compliment_list,user_id,compliment_writer,...,compliment_plain,compliment_hot,average_stars,compliment_funny,compliment_note,cool,name,friends,compliment_cute,fans
0,2007-01-25 16:47:26,467,55,180,1259,65,2007,18,qVc8ODYU5SZjKXVBgXdI7w,239,...,844,250,3.91,467,232,5994,Walker,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",56,267
1,2009-01-25 04:35:42,3131,184,1946,13066,264,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...",251,j14WgRoU_-2ZE1aw1dXrJg,1521,...,7054,1145,3.74,3131,1847,27281,Daniel,"ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",157,3138
2,2008-07-25 10:41:00,119,10,18,1010,13,"2009,2010,2011,2012,2013",3,2WnXYQFK0hXEoTxPtV2zvg,35,...,96,89,3.32,119,66,1003,Steph,"LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",17,52
3,2005-11-29 04:38:33,26,1,9,330,4,"2009,2010,2011",2,SZDeASXq7o05mMNLshsdIA,10,...,16,24,4.27,26,12,299,Gwen,"enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",6,28
4,2007-01-05 19:40:59,0,0,0,15,1,NaN,0,hA5lMy-EnncsH4JoR-hFGQ,0,...,1,1,3.54,0,1,7,Karen,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",0,1


In [53]:
user.isna().any()

yelping_since         False
compliment_cool       False
compliment_profile    False
compliment_photos     False
funny                 False
compliment_more       False
elite                  True
compliment_list       False
user_id               False
compliment_writer     False
useful                False
review_count          False
compliment_plain      False
compliment_hot        False
average_stars         False
compliment_funny      False
compliment_note       False
cool                  False
name                   True
friends               False
compliment_cute       False
fans                  False
dtype: bool

In [ ]:
# no na values in user_id

In [55]:
review = pd.read_csv('/content/drive/MyDrive/YelpRecommender/Data/review.csv')

In [58]:

for i in range(len(review)):
  if not isinstance(review['text'][i],str):
    print(type(review['text'][i]))


# we don't have any text review of different datatype

In [62]:
review.stars.isna().any()

False

In [63]:
review.stars.unique()

array([3., 5., 4., 1., 2.])

In [ ]:
# stars variable also has just 5 values. and no na values

In [68]:
review.loc[2201840,['cool','useful']] = 0
# have a single -1 value needed to be handles

In [69]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date         object 
 1   user_id      object 
 2   useful       int64  
 3   cool         int64  
 4   text         object 
 5   funny        int64  
 6   review_id    object 
 7   business_id  object 
 8   stars        float64
dtypes: float64(1), int64(3), object(5)
memory usage: 480.0+ MB


In [70]:
# Let us keep only the businesses in new orleans

In [73]:
new_orleans_review = pd.merge(review,new_orleans_business[['business_id','state']],on='business_id',how='inner' )

In [80]:
new_orleans_review = new_orleans_review[new_orleans_review.columns[0:9]]

In [81]:
new_orleans_review.to_csv('/content/drive/MyDrive/YelpRecommender/Data/NewOrleans/review.csv')

In [82]:
## let's work on tips dataset

In [83]:
tip = pd.read_csv('/content/drive/MyDrive/YelpRecommender/Data/tip.csv')

In [84]:
tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   date              908915 non-null  object
 1   user_id           908915 non-null  object
 2   text              908910 non-null  object
 3   compliment_count  908915 non-null  int64 
 4   business_id       908915 non-null  object
dtypes: int64(1), object(4)
memory usage: 34.7+ MB


In [85]:
# all values are non null
# therefore we will just merge the dataframes based on business_id in business dataframe

In [86]:
new_orleans_tip = pd.merge(tip,new_orleans_business[['business_id','state']],on='business_id',how='inner' )

In [91]:
new_orleans_tip = new_orleans_tip[new_orleans_tip.columns[0:5]]

In [92]:
new_orleans_tip.to_csv('/content/drive/MyDrive/YelpRecommender/Data/NewOrleans/tip.csv')

In [93]:
# Last let's update the checkin dataset

In [94]:
checkin = pd.read_csv('/content/drive/MyDrive/YelpRecommender/Data/checkin.csv')

In [95]:
checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [101]:
new_orleans_checkin = pd.merge(checkin,new_orleans_business[['business_id','state']],on='business_id',how='inner' )

In [102]:
new_orleans_checkin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3257 entries, 0 to 3256
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  3257 non-null   object
 1   date         3257 non-null   object
 2   state        3257 non-null   object
dtypes: object(3)
memory usage: 101.8+ KB


In [103]:
new_orleans_checkin = new_orleans_checkin[new_orleans_checkin.columns[0:2]]

In [105]:
new_orleans_checkin.to_csv('/content/drive/MyDrive/YelpRecommender/Data/NewOrleans/checkin.csv')